In [1]:
#Previamente Instalar: transformers, datasets, librosa
%pip install datasets
%pip install transformers
%pip install librosa


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Realizamos los imports necesarios
import os
import logging
import librosa

from datasets import DatasetDict, load_dataset
from transformers import (
    Wav2Vec2FeatureExtractor
)


logging.basicConfig(
    format="%(asctime)s | %(levelname)s: %(message)s", level=logging.INFO
)

c:\Users\albii\OneDrive\Escritorio\ALBA\4\toDataset\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PreTrainedFeatureExtractor = "SequenceFeatureExtractor"

NUM_LABELS = 6

PROJECT_ROOT =  os.path.dirname(os.path.abspath('PrepareDataset.ipynb')) + '/'

In [4]:
model_id = "facebook/hubert-base-ls960"

#cargamos el extractor de las caracteristicas principales de los audios
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_id)

In [5]:
#preparamos la configuracion para generar los datasets
dataset_config = {
  "LOADING_SCRIPT_FILES": os.path.join(PROJECT_ROOT, "src/data/crema.py"),
  "CONFIG_NAME": "clean",
  "DATA_DIR": os.path.join(PROJECT_ROOT, "data/archive.zip"),
  "CACHE_DIR": os.path.join(PROJECT_ROOT, "cache_crema"),
}

ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)

KeyboardInterrupt: 

In [ ]:
#Convertimos nuestra base de datos en arrays
ds = ds.map(
        lambda x: {
            "array": librosa.load(x["file"], sr=16000, mono=False)[0]
        }
    )


In [ ]:
'''Con esta funcion:
    - Leemos la matriz del audio y se extraen las caracteristicas de ella con feature_extractor
    - Crea una nueva característica llamada "labels" y su valor es el mismo que batch["label"]
'''

INPUT_FIELD = "input_values" 
LABEL_FIELD = "labels"

#IMPORTANTE: establecer la misma sr que el modelo que estamos utilizando(Facebook), en este caso 16KHz
def prepare_dataset(batch, feature_extractor: PreTrainedFeatureExtractor):
    audio_arr = batch["array"]
    input = feature_extractor(
        audio_arr, 
        sampling_rate=16000, 
        padding=True, 
        return_tensors="pt"
    )

    batch[INPUT_FIELD] = input.input_values[0]
    batch[LABEL_FIELD] = batch["label"]  # colname MUST be labels as Trainer will look for it by default

    return batch


In [ ]:
# Convertimos todas las etiquetas de texto en numeros 0-5
ds = ds.class_encode_column("label")

In [ ]:
# APPLY THE DATA PREP USING FEATURE EXTRACTOR TO ALL EXAMPLES
ds = ds.map(
    prepare_dataset,
    fn_kwargs={"feature_extractor": feature_extractor}
    # num_proc=2,
)
logging.info("Finished extracting features from audio arrays.")

In [ ]:
#Guardamos el dataset en el directorio ''datasets''
ds.save_to_disk(PROJECT_ROOT + 'datasets')